In [2]:
!pip install -q jsonformer datasets mwparserfromhell beautifulsoup4


[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: pip install --upgrade pip


In [ ]:
from datasets import load_from_disk, load_metric, load_dataset

datasets = load_dataset("wikipedia", language="simple", date="20220301")

In [23]:
from bs4 import BeautifulSoup
import requests

def get_wikidatapage(url):
    # HTML content
    response = requests.get(url)

    # Parse the HTML content
    soup = BeautifulSoup(response.text, 'html.parser')

    # Find the <script> tag with type="application/ld+json"
    script_tag = soup.find('script', {'type': 'application/ld+json'})

    if script_tag:
        # Extract the JSON content from the script tag
        json_content = script_tag.string

        # Parse the JSON content
        import json
        data = json.loads(json_content)

        # Extract the Wikidata link
        wikidata_link = data.get('sameAs')

        if wikidata_link:
            print("Wikidata link:", wikidata_link)
            return wikidata_link
        else:
            print("Wikidata link not found.")
    else:
        print("Script tag not found.")

get_wikidatapage(url)

Wikidata link: http://www.wikidata.org/entity/Q735


In [25]:
datasets['train'][5]

{'id': '12',
 'url': 'https://simple.wikipedia.org/wiki/Autonomous%20communities%20of%20Spain',
 'title': 'Autonomous communities of Spain',
 'text': 'Spain is divided in 17 parts called autonomous communities. Autonomous means that each of these autonomous communities has its own executive, legislative judicial powers. These are similar to, but not the same as, states in the United States of America, for example.\n\nSpain has fifty smaller parts called provinces. In 1978 these parts came together, making the autonomous communities. \nBefore then, some of these provinces were together but were broken. The groups that were together once before are called "historic communities": Catalonia, Basque Country, Galicia and Andalusia.\n\nThe Spanish language is the sole official language in every autonomous community but six, where Spanish is co-official with other languages, as follows:\n Catalonia: Catalan and Occitan\n Valencian Community: Catalan (also called Valencian there)\n Balearic Isl

In [ ]:
from jsonformer import Jsonformer
from transformers import AutoModelForCausalLM, AutoTokenizer

model = AutoModelForCausalLM.from_pretrained("databricks/dolly-v2-12b")
tokenizer = AutoTokenizer.from_pretrained("databricks/dolly-v2-12b")

json_schema = {
    "type": "object",
    "properties": {
        "name": {"type": "string"},
        "age": {"type": "number"},
        "is_student": {"type": "boolean"},
        "courses": {
            "type": "array",
            "items": {"type": "string"}
        }
    }
}

prompt = "Generate a person's information based on the following schema:"
jsonformer = Jsonformer(model, tokenizer, json_schema, prompt)
generated_data = jsonformer()

print(generated_data)